In [ ]:
import numpy as np # math library, simulation of random variables
import matplotlib.pyplot as plt
import seaborn as sns # more fancy stats visualization
from scipy import stats # stats library
from scipy import optimize # optimization library 
import statsmodels.api as sm # statistical modeling library

In this notebook, we run simulation about a hypothetical clinical trial scenario, in which we are comparing two drugs, with respective (unknown) probability of success $p_1$ and $p_2$. We are interested in testing $$\mathcal{H}_0 : (p_1 = p_2) \ \ \text{against  } \ \ \mathcal{H}_1 : (p_1 \neq p_2)$$ based on the observation of the efficacy of the treatments for $n$ patients who received treatment $1$, $X=(X_1,\dots,X_n)$ and $n$ patients who received treatment $2$, $Y=(Y_1,\dots,Y_n)$. 

All the observations are assumed to be independent with $X_i \sim \mathcal{B}(p_1)$ and $Y_{j} \sim \mathcal{B}(p_2)$. 

We will explore numerically the properties of two tests seen in class: the Wald test and the (Generalized) Likelihood ratio test. 

# Generating Bernoulli data

In [ ]:
## Generating data 
n = 100 # number of patients in each group
p1 = 0.8 # probability of success of treatment 1
p2 = 0.9 # probability of success of treatment 2

rng = np.random.RandomState(100)
X = (rng.random(n)<p1).astype(int)
print("average efficacy of treatment 1 is", np.mean(X))

Y = (rng.random(n)<p2).astype(int)
print("average efficacy of treatment 2 is", np.mean(Y))

# Wald test 

We recall the principle of the Wald test. If our test is about a parameter being equal to a certain value (here $p_2-p_1 = 0$), propose an estimator of this parameter, derive its asymptotic distribution (with a data-dependent variance), and deduce a Gaussian test statistic for your test. In our case, the estimator is $\hat{p}_{2,n} - \hat{p}_{1,n}$ where $\hat{p}_{1,n} = \frac{1}{n}\sum_{i=1}^{n} X_i$ is the average efficacy for patients who received treatment 1, and $\hat{p}_{2,n} = \frac{1}{n}\sum_{i=1}^{n} Y_i$ is the average efficacy for patients who received treatment 2. 

In class we proved that, for any $p_1,p_2$, 
$$\frac{\hat{p}_{2,n} - \hat{p}_{1,n} - (p_1 - p_2)}{\sqrt{\frac{\hat{p}_{1,n}(1-\hat{p}_{1,n})}{n} + \frac{ \hat{p}_{2,n}(1- \hat{p}_{2,n})}{n}}}$$ 
converges in distribution to a $\mathcal{N}(0,1)$ random variable. In particular, under $\mathcal{H}_0$ (that is, when $p_1 = p_2$), the test statistic 
$$T_n = \frac{\hat{p}_{2,n} - \hat{p}_{1,n}}{\sqrt{\frac{\hat{p}_{1,n}(1-\hat{p}_{1,n})}{n} + \frac{ \hat{p}_{2,n}(1- \hat{p}_{2,n})}{n}}}$$
is asymptotically a standard Gaussian. The Wald test with threshold $t$ is given by 
$$D(X) = \mathbb{1}(|T_n| > t)$$

**Complete the code below with the threshold that guarantees an asymptotic test of level $\alpha$.**


In [ ]:
def Wald_Test_Statistic(X,Y):
    # computing Tn
    n1 = len(X)
    n2 = len(Y)
    p1_hat = np.mean(X)
    p2_hat = np.mean(Y)
    var1 = p1_hat*(1-p1_hat)
    var2 = p2_hat*(1-p2_hat)
    if (var1 ==0)&(var2==0):
        # avoid division by zero
        return 0
    else:
        return (p2_hat - p1_hat)/np.sqrt(var1/n1 + var2/n2) 

def Wald_test(X,Y,alpha=0.05):
    # return 1 if H0 is rejected, 0 otherwise
    test_statistic = Wald_Test_Statistic(X,Y)
    threshold = ### TO BE COMPLETED
    decision = 0
    if (np.abs(test_statistic)>threshold):
        decision = 1
    return decision 

### Running the test

The cell below is running the Wald for given values of $p_1$, $p_2$ and the number of patients $n$ on simulated Bernoulli data. 

**You can try different values of the parameters and different seeds (corresponding to different observations, that can be seen as observations for different groups of patients), and see when the test is rejecting H0**

In [ ]:
n = 100 # number of patients in each group
p1 = 0.8
p2 = 0.9

rng = np.random.RandomState(2003)
X = (rng.random(n)<p1).astype(int)
print("average efficacy of treatment 1 is", np.mean(X))
Y = (rng.random(n)<p2).astype(int)
print("average efficacy of treatment 2 is", np.mean(Y))

print("decision of the Wald test:", Wald_test(X,Y))

### Distribution of the test statistic

A good test should use a test statistic whose *distribution* is significantly different from the null to an alternative, so as to make it very unlikely to reject when $\mathcal{H}_0$ is true (small type one error) and very likely to reject when $\mathcal{H}_1$ is true (large power / small type II error).  

Under $\mathcal{H}_1$, that is when $p_1 \neq p_2$, the statistic $T_n$ satisfies 
$$T_n = Z_n + \frac{p_1 - p_2}{\sqrt{\frac{\hat{p}_{1,n}(1-\hat{p}_{1,n})}{n} + \frac{ \hat{p}_{2,n}(1- \hat{p}_{2,n})}{n}}}$$
where $Z_n$ converges in distribution to a $\mathcal{N}(0,1)$, so the distribution of $T_n$ is (approximately) 
$$T_n \simeq \mathcal{N}\left( \frac{(p_1 - p_2)\sqrt{n}}{\sqrt{p_1(1-p_1) + (p_2(1-p_2)}}, 1\right)$$
whereas under $\mathcal{H}_0$, 
$$T_n \simeq \mathcal{N}\left(0, 1\right)\;.$$

To verify the quality of these approximations, the code below compares a (normalized) histogram of the empirical distribution of the test statistic T_n with the density of the corresponding Gaussian random variable. The empirical distribution is estimated based on $M$ simulations (i.e. it requires to run $M$ times the test, that is to generate $M$ times $2n$ Bernoulli random variables).  

**Explore how the quality of the approximation evolves with $n$, both under $H_0$ and $H_1$**

In [ ]:
p1=0.7 
p2=0.7

alpha_0 = 0.05 # desired type I error 

M = 1000 # number of simulations 
n = 30 

X_table = (np.random.random(size=(M,n))<p1).astype(int)
Y_table = (np.random.random(size=(M,n))<p2).astype(int)

statistic_values = np.zeros(M)

for m in range(M):
    X_small = X_table[m,range(n)]
    Y_small = Y_table[m,range(n)]
    statistic_values[m]=Wald_Test_Statistic(X_small,Y_small)

mean_value = np.sqrt(n)*(p2-p1)/(np.sqrt(p1*(1-p1)+p2*(1-p2)))
xplot = np.linspace(mean_value -5,mean_value+5,num=100)

law = stats.norm(np.sqrt(n)*(p2-p1)/(np.sqrt(p1*(1-p1)+p2*(1-p2))),1)
plt.hist(statistic_values,density=True,bins=15)
plt.plot(xplot,law.pdf(xplot))

The cell below visualizes the density of the asymptotic distribution of $T_n$ when $p_1=p_2$ and for the point $p_1=0.8,p_2=0.9$ in the alternative as well as the boundaries of the rejection region (which is outside the two vertical lines).  

**Where do we read the power (at this specific alternative) on the plot? You can play with the parameters and see how the power evolves with the sample size, the gap between p1 and p2 and the type I error alpha**

In [ ]:
p1=0.8
p2=0.9

n = 100
alpha = 0.05

mean_h1 = np.sqrt(n)*(p1-p2)/(np.sqrt(p1*(1-p1)+p2*(1-p2)))

law1=stats.norm
law2=stats.norm(mean_h1,1)

thres = law1.ppf(1-alpha/2)

xplot = np.linspace(-np.abs(mean_h1)-3,np.abs(mean_h1)+3,num=200)
plt.plot(xplot,law1.pdf(xplot),label="$T_n$ under H0")
plt.plot(xplot,law2.pdf(xplot),label="$T_n$ under H1")
plt.axvline(thres, 0, 1, c='black', linewidth = 1)
plt.axvline(-thres, 0, 1, c='black', linewidth = 1)
plt.legend()
plt.show()


# Approximation of the type I and type II error for a fixed sample size

The above calculation of power is purely asymptotic and uses some approximations, but we can use some simulation to actually *estimate* the type I error and power. Indeed, for $p_1=p_2=p$, 
$$\alpha((p,p)) = \mathbb{P}_{(p,p)} (D_n(X,Y) = 1) = \mathbb{E}_{(p,p)}[D_n(X,Y)]$$
which can be estimated using an empirical average. 

Given the simulation of $M$ experiments, giving the observations $(X^{m},Y^{m})$ for $m=1,\dots,M$, we can propose the estimate 
$$\widehat{a}_M = \frac{1}{M}\sum_{m=1}^{M} D_n(X^{m},Y^{m})\;.$$
That is, we run $M$ times the test on $M$ different dataset of patients, and report the average number of rejections. 

**Pick a value of $p$. How many samples seem necessary to get the type I error below $0.05$ for $(p,p) \in \mathcal{H}_0$?**

In [ ]:
p=0.8 # common value of p1 and p2 for an element in H0
alpha_0 = 0.05 # desired type I error 

M = 1000 # number of simulations 
n_values = np.array([30,50,100,200,300,500,1000]) # number of patients for which we want to estimate the type I error  
n_max =np.max(n_values)
n_nb = len(n_values)

X_table = (np.random.random(size=(M,n_max))<p).astype(int)
Y_table = (np.random.random(size=(M,n_max))<p).astype(int)

alphas = np.zeros(n_nb)

for i in range(n_nb):
    n = n_values[i] 
    for m in range(M):
        X_small = X_table[m,range(n)]
        Y_small = Y_table[m,range(n)]
        decision=Wald_test(X_small,Y_small,alpha_0)
        alphas[i]=alphas[i]+decision
    alphas[i]=alphas[i]/M

plt.plot(n_values,alphas)
plt.axhline(alpha_0,0, 1, c='black', linewidth = 1)
plt.title("empirical type I error at p1=p2={} as a function of the sample size".format(p))
plt.show()


You can do similar simulations for the power, estimated as 
$$\frac{1}{M}\sum_{m=1}^{M} D_n(X^{m},Y^{m})\;.$$
(fraction of rejections)when the data is simulated under an instance $(p_1,p_2)$ in the alternative. 

**How many samples seem necessary to get a power 0.8 under the alternative $p_1=0.8$, $p_2=0.9$?**

# Likelihood-Ratio Test

For Bernoulli data, the log-likelihood ratio statistic associated to the test 
$\mathcal{H}_0 : (p_1 = p_2)$ against $\mathcal{H}_1 : (p_1\neq p_2)$ is 

$$\log \widetilde{\Lambda}(X,Y) = \log\frac{\sup_{p_1 \in [0,1]} p_1^{S_1}(1-p_1)^{n-S_1}\sup_{p_2 \in [0,1]} p_2^{S_2}(1-p_2)^{n-S_2}}{\sup_{p \in [0,1]} p^{S_1}(1-p)^{n-S_1}p^{S_2}(1-p)^{n-S_2}} $$
where $S_1$ is the number of successes of treatment 1 among the $n$ patients who received it and $S_2$ is the number of successes of treatment 2 among the $n$ patients who received it.

Introducing the estimates $\hat{p}_{1,n}$, $\hat{p}_{2,n}$ and $\widehat{p}_{n} = \frac{1}{2n}\sum_{i=1}^{n}(X_i+Y_i)$ (the average efficacy across all patients), we have (dropping the index $n$ for readability)

\begin{align}
\log \widetilde{\Lambda}(X,Y) & = \log\frac{\hat{p}_1^{S_1}(1-\hat{p}_1)^{n-S_1}\hat{p}_2^{S_2}(1-\hat{p}_2)^{n-S_2}}{\hat{p}^{S_1}(1-\hat{p})^{n-S_1}\hat{p}^{S_2}(1-\hat{p})^{n-S_2}} \\
& = n \left(\hat{p}_1 \log \frac{\hat{p}_1}{\hat{p}} + (1-\hat{p}_1) \log \frac{1-\hat{p}_1}{1-\hat{p}}\right) + n \left(\hat{p}_2 \log \frac{\hat{p}_2}{\hat{p}} + (1-\hat{p}_2) \log \frac{1-\hat{p}_2}{1-\hat{p}}\right) \\
& = n \mathrm{kl}(\hat{p}_1,\hat{p}) + n \mathrm{kl}(\hat{p}_2,\hat{p})
\end{align}

where we introduce the KL divergence between two Bernoulli distributions with mean $p$ and $q$ as $\mathrm{kl}(p,q) = p\log\frac{p}{q} + (1-p)\log \frac{1-p}{1-q}$

**Complete the function below to compute the Log-Likelihood Ratio**

In [ ]:
eps = 1e-15    

def kl(p, q):
    """Kullback-Leibler divergence for Bernoulli distributions."""
    p = min(max(p, eps), 1-eps)
    q = min(max(q, eps), 1-eps)
    return p*np.log(p/q) + (1-p)*np.log((1-p)/(1-q))

def Log_Likelihood_Ratio(X,Y): 
    ## TO BE COMPLETED

A LRT test is of the form 
$$D(X,Y) = \mathbb{1}\left(\log\widetilde{\Lambda}(X,Y) > t\right)$$
for some threshold $t$. 

**Complete the function below to implement a LRT guaranteed to have an asymptotic type I error of $\alpha$ using Wilk's theorem.**

In [ ]:
def LRT_test(X,Y,alpha=0.05):
    test_statistic = ## TBC
    threshold = ## TBC
    decision = 0
    if (test_statistic>threshold):
        decision = 1
    return decision 

### Running the test 

As before, you can run the test with varying parameters, and also compare its outcome to the Wald test.

In [ ]:
n = 100 # number of patients in each group
p1 = 0.64
p2 = 0.7

rng = np.random.RandomState(2025)
X = (rng.random(n)<p1).astype(int)
print("average efficacy of treatment 1 is", np.mean(X))
Y = (rng.random(n)<p2).astype(int)
print("average efficacy of treatment 2 is", np.mean(Y))

print("decision of the LRT test:", LRT_test(X,Y))
print("decision of the Wald test:", Wald_test(X,Y))

## Distribution of the test statistic 

The code below permit to check that under $\mathcal{H}_0$, the statistic $2\log\widetilde{\Lambda}(X,Y)$ seem to follow a chi-square distribution with one degree of freedom, at least when $n$ is large enough. 

In [ ]:
p1=0.7 
p2=0.7

alpha_0 = 0.05 # desired type I error 

M = 1000 # number of simulations 
n = 100 

X_table = (np.random.random(size=(M,n))<p1).astype(int)
Y_table = (np.random.random(size=(M,n))<p2).astype(int)

statistic_values = np.zeros(M)

for m in range(M):
    X_small = X_table[m,range(n)]
    Y_small = Y_table[m,range(n)]
    statistic_values[m]=2*Log_Likelihood_Ratio(X_small,Y_small)

mean_value = np.sqrt(n)*(p2-p1)/(np.sqrt(p1*(1-p1)+p2*(1-p2)))
xplot = np.linspace(0,mean_value +5,num=100)

thres = stats.chi2(1).ppf(1-alpha)

law = stats.chi2(1)
plt.hist(statistic_values,density=True,bins=15)
plt.plot(xplot,law.pdf(xplot))
plt.axvline(thres, 0, 1, c='black', linewidth = 1)

You can similarly explore the distribution of $2\log\widetilde{\Lambda}(X,Y)$ under some element of $\mathcal{H}_1$.

In [ ]:
p1=0.7 
p2=0.9

alpha_0 = 0.05 # desired type I error 

M = 1000 # number of simulations 
n = 100 

X_table = (np.random.random(size=(M,n))<p1).astype(int)
Y_table = (np.random.random(size=(M,n))<p2).astype(int)

statistic_values = np.zeros(M)

for m in range(M):
    X_small = X_table[m,range(n)]
    Y_small = Y_table[m,range(n)]
    statistic_values[m]=Log_Likelihood_Ratio(X_small,Y_small)

mean_value = np.sqrt(n)*(p2-p1)/(np.sqrt(p1*(1-p1)+p2*(1-p2)))
xplot = np.linspace(0,mean_value +5,num=100)

thres = stats.chi2(1).ppf(1-alpha)

law = stats.chi2(1)
plt.hist(statistic_values,density=True,bins=15)
plt.plot(xplot,law.pdf(xplot))
plt.axvline(thres, 0, 1, c='black', linewidth = 1)

## Estimation of the test characteristics

You can now compare the type I error and the power of the Wald Test and the Likelihood Ratio Test for different values of $p_1$ and $p_2$ in $H_0$ and $H_1$.